In [1]:
import numpy as np
import pandas as pd
import sys
import psutil

from sklearn.model_selection import train_test_split
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, History 

Using TensorFlow backend.


In [2]:
df = pd.read_csv('wine_multiclass_imbalanced.csv',index_col=0)

# Detecting no of classes 

In [3]:
no_of_classes = df['class'].nunique()

In [4]:
df['class'].value_counts()

2    71
1    59
3    48
Name: class, dtype: int64

In [5]:
names_of_classes = list(df['class'].unique())

# Data Pre Processing

In [6]:
Y = df['class']
dummy_Y = pd.get_dummies(Y,sparse=True)

In [7]:
pre_processed_df = pd.merge(df,dummy_Y,left_index=True,right_index=True)

In [8]:
# pre_processed_df.drop(['class'],axis=1,inplace=True)

# Data Splitting

In [9]:
def split_train_test(df,label_cols,test_size=0.2):    
    X = df[df.columns.difference(label_cols)]
    y = df[label_cols]
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=test_size, random_state=42)
    return train_x, test_x, train_y, test_y

In [10]:
x_train, x_test, y_train, y_test = split_train_test(df=pre_processed_df,label_cols=names_of_classes)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

In [11]:
x_train.shape, x_valid.shape, x_test.shape

((106, 14), (36, 14), (36, 14))

In [12]:
y_train.shape, y_valid.shape, y_test.shape

((106, 3), (36, 3), (36, 3))

In [13]:
x_train_model = x_train[x_train.columns.difference(['class'])]

In [14]:
x_valid_model = x_valid[x_valid.columns.difference(['class'])]

In [15]:
x_test_model = x_test[x_test.columns.difference(['class'])]

# Balancing Factors

In [16]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(x_train['class']),x_train['class'])

In [17]:
class_weights

array([1.03921569, 0.90598291, 1.07070707])

# Implementing Keras Model

In [18]:
def get_search_space():    
    space = {'num_layers': hp.choice('num_layers',['one_hidden', 'two_hidden']),
                'units1': hp.choice('units1', [32, 64, 128, 256,512]),
                'units2': hp.choice('units2', [32, 64, 128, 256,512]),
                'dropout1': hp.uniform('dropout1', .25,.75),
                'dropout2': hp.uniform('dropout2',  .25,.75),
                'batch_size' : hp.choice('batch_size', [16,32,64,128]),
                'nb_epochs' :  500,
                'optimizer': hp.choice('optimizer',['rmsprop', 'adam', 'nadam','sgd']),
                'activation': hp.choice('activation',['relu','sigmoid']),
                'early_stop_rounds': hp.choice('early_stop_rounds',[10,20,30,40,50]),
            }
    return space

In [19]:
def create_model(params):    
    x_train_temp = x_train_model.copy() 
    y_train_temp = y_train.copy()
    model = Sequential()
    model.add(Dense(params['units1'], input_shape=(x_train_temp.shape[1],)))
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout1']))
    if(params['num_layers'] == 'two_hidden'):
        model.add(Dense(params['units2']))
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dropout2']))
    model.add(Dense(y_train_temp.shape[1]))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer=params['optimizer'])
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=params['early_stop_rounds'])
    terminate_nan = keras.callbacks.TerminateOnNaN()
    history = History()
    model.fit(x_train_temp, y_train_temp,
              batch_size=params['batch_size'],
              epochs=500,
              callbacks=[early_stop, terminate_nan, history],
              verbose=0,
              validation_data=(x_valid_model,y_valid),
             class_weight=class_weights) 
    [loss, acc] = model.evaluate(x_valid_model,y_valid, verbose=0)
    global num
    mem = psutil.virtual_memory()
    if(np.isnan(acc)):
        print("{}) Validation set Accuracy: NaN".format(num),"\tAvailable Mem:",(mem.available/1024)/1024,"mb")
        num = num + 1
        return {'loss': np.inf, 'status': STATUS_OK, 'params': params}
    print("{}) Validation set Accuracy: {:7.2f}".format(num,acc*100),"\tAvailable Mem:",(mem.available/1024)/1024,"mb")
    num = num + 1
    return {'loss': -acc, 'status': STATUS_OK, 'params': params}

In [20]:
def train_best_model(best_params):   
    print('Training the best selected model...') 
    x_train_temp = x_train_model.copy() 
    y_train_temp = y_train.copy()
    model = Sequential()
    model.add(Dense(best_params['units1'], input_shape=(x_train_temp.shape[1],)))
    model.add(Activation(best_params['activation']))
    model.add(Dropout(best_params['dropout1']))
    if(best_params['num_layers'] == 'two_hidden'):
        model.add(Dense(best_params['units2']))
        model.add(Activation(best_params['activation']))
        model.add(Dropout(best_params['dropout2']))
    model.add(Dense(y_train_temp.shape[1]))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer=best_params['optimizer'])
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=best_params['early_stop_rounds'])
    history = History()
    model.fit(x_train_temp, y_train_temp,
              batch_size=best_params['batch_size'],
              epochs=500,
              callbacks=[early_stop, history],
              verbose=0,
              validation_data=(x_valid_model,y_valid),
             class_weight=class_weights) 

    return model

In [25]:
num= 1
trials=Trials()
space = get_search_space()
print("Selecting the best network architecture specifically for your data...")
best = fmin(create_model, space, algo=tpe.suggest, max_evals=2, trials=trials)
best_trials_temp = trials.best_trial['result'] 
best_model_temp = train_best_model(best_trials_temp['params']) 
# scaled_feature_df = pd.concat([x_train,x_valid,x_test])
# label_df = pd.concat([y_train,y_valid,y_test])
# pred_df = make_predictions(model=best_model_temp,df=scaled_feature_df)
# output_df = pd.merge(input_df,pred_df['predictions'].to_frame(),left_index=True,right_index=True)
# return best_model_temp, output_df, test_arg_processed

Selecting the best network architecture specifically for your data...
1) Validation set Accuracy:   52.78 	Available Mem: 2306.81640625 mb
2) Validation set Accuracy:   94.44 	Available Mem: 2281.33984375 mb
Training the best selected model...


In [55]:
predicted = best_model_temp.predict_classes(x_test_model)

In [60]:
predicted

array([0, 0, 2, 0, 0, 0, 2, 2, 1, 2, 0, 2, 0, 2, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 2, 2, 0, 0, 0])

In [61]:
# predicted1 = np.argmax(predicted, axis=1)

In [62]:
y_pred = pd.get_dummies(predicted)

In [64]:
y_pred.columns = y_test.columns

In [73]:
y_pred.index = x_test_model.index

In [75]:
a = pd.merge(x_test_model,y_pred,left_index=True,right_index=True)

In [78]:
reversed_Y_col = reverse_dummies(a[y_pred.columns],'class')

     1  2  3
19   1  0  0
45   1  0  0
140  0  0  1
30   1  0  0
67   1  0  0


In [36]:
best_model_temp.evaluate(x_test_model,y_test)

36/36 [==============================] - 0s 105us/step


[0.2653280339307255, 0.8333333333333334]

In [37]:
y_pred.columns = y_test.columns

In [30]:
def reverse_dummies(df,label_col):
    print(df.head())
    predict_col = pd.DataFrame(df.idxmax(axis=1),columns=[label_col + '_prediction'])
    return predict_col

In [49]:
y_pred_reversed = reverse_dummies(x_test_model['predictions'].to_frame(),'class')

     predictions
19             0
45             0
140            2
30             0
67             0


In [51]:
x_test_model['predictions']

19     0
45     0
140    2
30     0
67     0
16     0
119    2
174    2
109    1
141    2
24     0
150    2
41     0
118    2
15     0
111    1
113    1
82     1
9      0
114    1
18     0
66     0
60     0
169    1
171    1
164    0
117    1
65     1
90     0
55     0
29     0
128    2
145    2
31     0
12     0
42     0
Name: predictions, dtype: int64

In [50]:
y_pred_reversed

,class_prediction
19,predictions
45,predictions
140,predictions
30,predictions
67,predictions
16,predictions
119,predictions
174,predictions
109,predictions
141,predictions
